In [13]:

import requests
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json

# Load environment variables from .env file
load_dotenv()

#Henter API-key/Client-id fra .env filen
client_id = os.getenv("client_id")

with open('../data/Byer.json', 'r') as f:
    byer = json.load(f)

byerAsString = ','.join(byer.values())

element_id = "mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)" # Temperatur, nedbør og vindhastighet
start_dato = "2022-01-01" # Startdato
slutt_dato = "2023-01-02" # Sluttdato


# Define endpoint and parameters
endpoint ='https://frost.met.no/observations/v0.jsonld'
parameters = {
    "sources": byerAsString, # Bruker værstasjoner fra Byer.json, Oslo og Trondheim
    "elements": element_id, 
    "referencetime": f"{start_dato}/{slutt_dato}",
    "timeoffsets": "default",
    "timeresolutions": "P1D",  # Daglige verdier
    "levels": "default"
}
print(f"Parametere: {parameters}")

# Sender forespørsel til Frost API
response = requests.get(endpoint, params=parameters, auth=(client_id, ''))

# Sjekker om forespørselen var vellykket
if response.status_code != 200:
    print(f"Error! Status code: {response.status_code}")
    print(f"Message: {response.json().get('error', {}).get('message', 'Ingen melding gitt')}")
    print(f"Reason: {response.json().get('error', {}).get('reason', 'Ingen årsak gitt')}")
    exit()

# Henter JSON-data
json_data = response.json()
if 'data' not in json_data:
    print("Ingen data funnet i forespørselen.")
    print(json_data)
    exit()

# Prosesserer data til en DataFrame
data = json_data['data']
rows = []  # Liste for å samle alle rader
for item in data:
    for observation in item['observations']:
        row = {
            'sourceId': item['sourceId'].split(':')[0],  # Fjern ':0' fra sourceId
            'referenceTime': item['referenceTime'],
            'elementId': observation['elementId'],
            'value': observation['value'],
            'unit': observation['unit'],
            'codequality': observation.get('qualityCode', '')  # Henter kvalitetskoden
        }
        rows.append(row)


# Opprett DataFrame fra rader
df = pd.DataFrame(rows)

# Legger til en ny kolonne for bynavn basert på sourceId
df['by'] = df['sourceId'].map({v: k for k, v in byer.items()})

df = df[['by','sourceId', 'referenceTime', 'elementId', 'value', 'unit', 'codequality']]

# Skriver DataFrame til en CSV-fil
data_fil = '../data/VærData.csv'
df.to_csv(data_fil, index=False)
print(f"Data lagret i {data_fil}")








Parametere: {'sources': 'SN68230,SN18700,SN50540', 'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)', 'referencetime': '2022-01-01/2023-01-02', 'timeoffsets': 'default', 'timeresolutions': 'P1D', 'levels': 'default'}
Data lagret i ../data/VærData.csv
